# Fairness Sandox Prototype

This rudimentary version of the framework will contain one dataset, one model, one bias mitigation algorithm, and a limited amount of validation metrics. Emphasis will be on modularity and abstraction so as to encourage more features in the future.

### Setup

Please run the code block below to install the necessary packages (if needed).

In [ ]:
%pip install aif360
%pip install fairlearn
%pip install imbalanced-learn
%pip install matplotlib
%pip install numpy
%pip install pandas
%pip install seaborn
%pip install scikit-learn

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_curve, auc

import fairlearn
import aif360

# Data + EDA

### Dataset Class

In [5]:
class Dataset:
    def __init__(self, short_name = '', path = '', cat_cols = [], num_cols = []):
        self.short_name = short_name
        self.path = path
        self.cat_cols = cat_cols
        self.num_cols = num_cols
        self.df = pd.read_csv(path, sep = ';')

### Popular Datasets

In [6]:
# each dataset is a dictionary where keys = short name, values = Dataset object

datasets = dict()

def add_dataset(dataset):
    if not isinstance(dataset, Dataset):
        print("Error! Please enter a valid Dataset object")
    else:
        if dataset.short_name not in datasets.keys():
            datasets[dataset.short_name] = dataset

In [7]:
# example - adding a dataset
path_adult_income = 'Datasets/adult.csv'
cat_cols = ['workclass', 'education','marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country','income']
num_cols = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']
adult_income = Dataset('adult_income', path_adult_income, cat_cols, num_cols)

add_dataset(adult_income)

# TODO - add more datasets

In [8]:
add_dataset(Dataset("student_mat", path='Datasets/student-mat.csv'))
add_dataset(Dataset("student_por", path='Datasets/student-por.csv'))

In [ ]:
# (potential) TODO: add functionality for missing data + 
#       encoding categorical features

### EDA

Feel free to use any/all of the following EDA functions and/or add your own!

In [ ]:
# take a peek at the first few data points
df_por = datasets['student_por'].df
df_por.head()

In [ ]:
def plot_counts(df, attr):
    if attr in df.columns:
        df[attr].value_counts(normalize=True).plot.barh()
    else:
        print("Error! Please enter a valid feature.")

# example
plot_counts(df_por, 'sex')

In [ ]:
def group_by_plot(df, attr1, attr2):
    for val in list(df[attr1].unique()):
        print(val)
        temp = df[df[attr1] == val]
        sns.displot(temp[attr2])

# example
group_by_plot(df_por, 'sex', 'G3')

In [ ]:
# add support for sns.pairplot, plt.scatter, sns.heatmap

# Model

### Model Selection

In [ ]:
print(42)

### Model Performance

In [ ]:
print(42)

# Bias Injection

### Bias Selection

### Post-Injection Visualization

# Fairness Intervention

# Evaluation